# Melanoma Classification - Data Investigation
Data Source: https://challenge2020.isic-archive.com/


## 1. Downloading the data
This step will take a while. Please be patient and do not aboard the project prematurely. We are downloading 25 GB. Please make sure you have enough space for this large amount of data.

If you have already previously downloaded the data, please move it into a folder called "data" in the directory of this project, so that the following steps can run without a problem.

The structure after this step will be as followed:
<pre>
data_investigation.ipynb
data
|---ISIC_2020_Training_Duplicates.csv
|---ISIC_2020_Training_GroundTruth_v2.csv
|---ISIC_2020_Training_JPEG.zip
</pre>
In the next step the images will be unzipped. Continue with step 2.

If you have already unzipped the data the structure should now look like this: 
<pre>
data_investigation.ipynb
data
|---ISIC_2020_Training_Duplicates.csv
|---ISIC_2020_Training_GroundTruth_v2.csv
|---ISIC_2020_Training_JPEG
    |---train
        |---ISIC_0099474.jpg
        |---...
</pre>
If you are at this data structure please continue with step 3.

In [1]:
import os
import requests 

path = 'data'
if not os.path.exists(path): 
  os.makedirs(path)
  print("Created new directory for data")

print('Downloading ISIC img data')
img_data = 'https://isic-challenge-data.s3.amazonaws.com/2020/ISIC_2020_Training_JPEG.zip'
r = requests.get(img_data, allow_redirects=True, stream = True)
with open("data/ISIC_2020_Training_JPEG.zip","wb") as file:
    for chunk in r.iter_content(chunk_size=1024):
         if chunk:
             file.write(chunk)

print('Downloading ISIC meta data')
meta_data = 'https://isic-challenge-data.s3.amazonaws.com/2020/ISIC_2020_Training_GroundTruth_v2.csv'
r = requests.get(meta_data, allow_redirects=True)
open('data/ISIC_2020_Training_GroundTruth_v2.csv', 'wb').write(r.content)

print('Downloading ISIC duplicate image list')
duplicate_list_data = 'https://isic-challenge-data.s3.amazonaws.com/2020/ISIC_2020_Training_Duplicates.csv'
r = requests.get(duplicate_list_data, allow_redirects=True)
open('data/ISIC_2020_Training_Duplicates.csv', 'wb').write(r.content)

Created new directory for data
Dowloading ISIC duplicate image list


11500

## 2. Unzipping downloaded data
The images are included in a .zip file. In order to work with the images, we have to unzip the data.

This step will also take quite a while. Be patient :-)

In [2]:
from zipfile import ZipFile

print("Unzipping Image Data")
with ZipFile("data/ISIC_2020_Training_JPEG.zip", 'r') as zObject:
    zObject.extractall(path="data/ISIC_2020_Training_JPEG")
print("Finished unzipping images")

Unzipping Image Data
Finished unzipping images


## 3. Filtering duplicates
The data includes some duplicates.These are stored in the file "data/ISIC_2020_Training_Duplicates.csv".
We do not want to train on duplicate data and will therefore remove these images from our training data.

In [3]:
import pandas as pd
import os

df_duplicates = pd.read_csv("data/ISIC_2020_Training_Duplicates.csv")
df_groundtruth = pd.read_csv("data/ISIC_2020_Training_GroundTruth_v2.csv")
img_path = 'data/ISIC_2020_Training_JPEG/train'

print("There are %i duplicate pictures in the data" %len(df_duplicates.index))
print("There are %i images in total in the meta-data file ('ground-truth')" %len(df_groundtruth.index))
print("There are %i images on the hard-drive" %len([name for name in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, name))]))

img_to_drop = df_duplicates.image_name_1.values.tolist()

print("Dropping duplicate images in ground truth")
df_groundtruth = df_groundtruth[df_groundtruth.image_name.isin(img_to_drop) == False]
df_groundtruth.to_csv("data/ISIC_2020_Training_GroundTruth_v2.csv")
print("After dropping the duplicates, there are %i images in total in the meta-data file ('ground-truth')" %len(df_groundtruth.index))

print("Deleting duplicate images from the hard-drive")
for img_name in img_to_drop:
    filename = img_path + "/" + img_name + ".jpg"
    try:
        os.remove(filename)
    except IOError:
        pass

print("There are now %i images on the hard-drive" %len([name for name in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, name))]))

There are 425 duplicate pictures in the data
There are 33126 images in total in the meta-data file ('ground-truth')
There are 33126 images on the hard-drive
Dropping images in ground truth
After dropping the duplicates, there are 32701 images in total in the meta-data file ('ground-truth')
Deleting duplicate images from the hard-drive
There are now 32701 images on the hard-drive


## 4. Train Test Validation split
In the next step, the data will be split into 3 parts. 

<b>Training Set</b>
* 80%
* Used to train the learning algorithm

<b>Validation</b>
* Part of training set that is held out (10% of overall data)
* Used to evaluate different algorithms and hyperparameter settings

<b>Test</b>
* 10%
* Used to test the model and estimate ist performance in application
* Will <b>not</b> be used for training under no circumstances

After this step, the folder structure will be as followed:
<pre>
data_investigation.ipynb
data
|---ISIC_2020_Training_GroundTruth_v2_train.csv
|---ISIC_2020_Training_GroundTruth_v2_test.csv
|---ISIC_2020_Training_GroundTruth_v2_valid.csv
|---train
    |---ISIC_0099484.jpg
    |---...
|---test
    |---ISIC_0077474.jpg
    |---...
|---valid
    |---ISIC_0089570.jpg
    |---...
</pre>

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import shutil

random_state = 42
df = pd.read_csv('data/ISIC_2020_Training_GroundTruth_v2.csv')

X = df.drop(columns = ['diagnosis', 'benign_malignant', 'target']).copy()
y = df[['diagnosis', 'benign_malignant', 'target']]

train_size=0.8 # 80% training set
X_train, X_remaining, y_train, y_remaining = train_test_split(X,y, 
                                                              train_size=train_size, 
                                                              random_state=random_state)

test_size = 0.5 # 50% of remaining 20% -> 10%
X_valid, X_test, y_valid, y_test = train_test_split(X_remaining, y_remaining, 
                                                    test_size=test_size, 
                                                    random_state=random_state)

print("The training set includes %i samples" %len(X_train.index))
print("The validation set includes %i samples" %len(X_valid.index))
print("The test set includes %i samples" %len(X_test.index))

print("Writing test train and validation data into seperate csv files")

train = pd.concat([X_train, y_train], axis=1, join='inner')
valid = pd.concat([X_valid, y_valid], axis=1, join='inner')
test = pd.concat([X_test, y_test], axis=1, join='inner')

train.to_csv("data/ISIC_2020_Training_GroundTruth_v2_train.csv")
valid.to_csv("data/ISIC_2020_Training_GroundTruth_v2_valid.csv")
test.to_csv("data/ISIC_2020_Training_GroundTruth_v2_test.csv")

print("Finished writing test train and validation data into seperate csv files")

train_img = X_train["image_name"].values.tolist()
test_img = X_test["image_name"].values.tolist()
valid_img = X_valid["image_name"].values.tolist()

print("Sorting images on hard-drive into seperate folders")

def sort_img(path, img_names):
    print("Coping %s images" %path)
    dest = 'data/' + path
    src = 'data/ISIC_2020_Training_JPEG/train'
    if not os.path.exists(dest): 
        os.makedirs(dest)
        print("Created new directory %s" %dest)
    for img_name in img_names:
        fname = src + "/" + img_name + ".jpg"
        shutil.move(fname, dest)

sort_img("train", train_img)
sort_img("test", test_img)
sort_img("valid", valid_img)

print("Finished sorting images on hard-drive into seperate folders")

print("Removing no longer needed files")
shutil.rmtree("data/ISIC_2020_Training_JPEG")
os.remove("data/ISIC_2020_Training_Duplicates.csv")
os.remove("data/ISIC_2020_Training_GroundTruth_v2.csv")
print("Finished removing no longer needed files")

The training set includes 26160 samples
The validation set includes 3270 samples
The test set includes 3271 samples
Writing test train and validation data into seperate csv files
Finished writing test train and validation data into seperate csv files
Sorting images on hard-drive into seperate folders
Created new directory data/train
Created new directory data/test
Created new directory data/valid
Finished sorting images on hard-drive into seperate folders
Removing no longer needed files
Finished removing no longer needed files
